# Optical Flow with Lucas-Kanade method Using | Python OpenCV


## What is the Optical Flow in motion tracking?

Optical flow is the pattern of apparent motion of image objects between two consecutive frames caused by the movemement of object or camera. It is 2D vector field where each vector is a displacement vector showing the movement of points from first frame to second.

#### Optical flow has many applications in areas like :

- Structure from Motion
- Video Compression
- Video Stabilization ...



OpenCV provides a single function, `cv2.calcOpticalFlowPyrLK()`.


**Syntax:** `cv.calcOpticalFlowPyrLK(prevImg, nextImg, prevPts, nextPts[, status[, err[, winSize[, maxLevel[, criteria[, flags[, minEigThreshold]]]]]]])`

**Parameters:**

- ***prevImg :–*** first 8-bit input image or pyramid constructed by buildOpticalFlowPyramid.
- ***nextImg :–*** second input image or pyramid of the same size and the same type as prevImg.
- ***prevPts :–*** vector of 2D points for which the flow needs to be found; point coordinates must be single-precision floating-point numbers.

- ***winSize :–*** size of the search window at each pyramid level.
- ***maxLevel :–***  0-based maximal pyramid level number; if set to 0, pyramids are not used (single level), if set to 1, two levels are used, and so on; if pyramids are passed to input then algorithm will use as many levels as pyramids have but no more than maxLevel.

- ***criteria :–*** parameter, specifying the termination criteria of the iterative search algorithm (after the specified maximum number of iterations criteria.maxCount or when the search window moves by less than criteria.epsilon.

- ***flags:-***
       
    - **OPTFLOW_USE_INITIAL_FLOW** uses initial estimations, stored in nextPts; if the flag is not set, then prevPts is copied to nextPts and is considered the initial estimate.
    - **OPTFLOW_LK_GET_MIN_EIGENVALS** use minimum eigen values as an error measure (see minEigThreshold description); if the flag is not set, then L1 distance between patches around the original and a moved point, divided by number of pixels in a window, is used as a error measure.


- ***minEigThreshold :-***	the algorithm calculates the minimum eigen value of a 2x2 normal matrix of optical flow equations (this matrix is called a spatial gradient matrix), divided by number of pixels in a window; if this value is less than minEigThreshold, then a corresponding feature is filtered out and its flow is not processed, so it allows to remove bad points and get a performance boost.




**Return:**

- ***nextPts :-*** output vector of 2D points (with single-precision floating-point coordinates) containing the calculated new positions of input features in the second image; when **OPTFLOW_USE_INITIAL_FLOW** flag is passed, the vector must have the same size as in the input.


- ***status :–*** output status vector (of unsigned chars); each element of the vector is set to 1 if the flow for the corresponding features has been found, otherwise, it is set to 0.


- ***err :–*** output vector of errors; each element of the vector is set to an error for the corresponding feature, type of the error measure can be set in flags parameter; if the flow wasn’t found then the error is not defined (use the status parameter to find such cases).


---

Here, we create a simple application which tracks some points in a video. To decide the points, we use cv2.goodFeaturesToTrack(). We take the first frame, detect some Shi-Tomasi corner points in it, then we iteratively track those points using Lucas-Kanade optical flow. For the function cv2.calcOpticalFlowPyrLK() we pass the previous frame, previous points and next frame. It returns next points along with some status numbers which has a value of 1 if next point is found, else zero. We iteratively pass these next points as previous points in next step. See the code below:



## Search for sift and pyramids...

In [1]:
import numpy as np
import cv2

cap = cv2.VideoCapture('slow.flv')

# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Create some random colors
color = np.random.randint(0,255,(100,3))

# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)

while(1):
    ret,frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]

    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    img = cv2.add(frame,mask)

    cv2.imshow('frame',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)

cv2.destroyAllWindows()
cap.release()

error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-52oirelq\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


# Dense Optical flow

Lucas-Kanade method computes optical flow for a sparse feature set (in our example, corners detected using Shi-Tomasi algorithm). OpenCV provides another algorithm to find the dense optical flow. It computes the optical flow vector for every pixel of the frame which may be responsible for its slow speed but leading to a better accurate result. There can be various kinds of implementations of dense optical flow. The example below will follow the Farneback method along with OpenCV.


### Franeback Method

- The first step is that the method approximates the windows of image frames by a quadratic polynomial with the help of the polynomial expansion transform. 

- Next, by observing how the polynomial transforms under the state of motion. i.e. to estimate displacement fields. Dense optical flow is computed, after a series of refinements.

### For OpenCV’s implementation

Below sample shows how to find the dense optical flow using above algorithm. We get a 2-channel array with optical flow vectors, (u,v). We find their magnitude and direction. We color code the result for better visualization. Direction corresponds to Hue value of the image. Magnitude corresponds to Value plane.  The strength of HSV is always set to a maximum of 255 for optimal visibility. See the code below:

#### Syntax: 
`cv.calcOpticalFlowFarneback(prev, next, flow, pyr_scale, levels, winsize, iterations, poly_n, poly_sigma, flags)`.

#### Parameters

- **prev :-**	first 8-bit single-channel input image.
- **next :-** second input image of the same size and the same type as prev.
- **flow :-**	computed flow image that has the same size as prev and type CV_32FC2.
- **pyr_scale :-**	parameter, specifying the image scale (<1) to build pyramids for each image; pyr_scale=0.5 means a classical pyramid, where each next layer is twice smaller than the previous one.

- **levels :-**	number of pyramid layers including the initial image; levels=1 means that no extra layers are created and only the original images are used.

- **winsize :-**	averaging window size; larger values increase the algorithm robustness to image noise and give more chances for fast motion detection, but yield more blurred motion field.

- **iterations :-**	number of iterations the algorithm does at each pyramid level.

- **poly_n :-**	size of the pixel neighborhood used to find polynomial expansion in each pixel; larger values mean that the image will be approximated with smoother surfaces, yielding more robust algorithm and more blurred motion field, typically poly_n =5 or 7.

- **poly_sigma :-**	standard deviation of the Gaussian that is used to smooth derivatives used as a basis for the polynomial expansion; for poly_n=5, you can set poly_sigma=1.1, for poly_n=7, a good value would be poly_sigma=1.5.

- **flags :-** operation flags that can be a combination of the following:
    - ***OPTFLOW_USE_INITIAL_FLOW*** uses the input flow as an initial flow approximation.
    - ***OPTFLOW_FARNEBACK_GAUSSIAN*** uses the Gaussian winsize×winsize filter instead of a box filter of the same size for optical flow estimation; usually, this option gives z more accurate flow than with a box filter, at the cost of lower speed; normally, winsize for a Gaussian window should be set to a larger value to achieve the same level of robustness.

[https://docs.opencv.org/master/dc/d6b/group__video__track.html#ga473e4b886d0bcc6b65831eb88ed93323](https://docs.opencv.org/master/dc/d6b/group__video__track.html#ga473e4b886d0bcc6b65831eb88ed93323)

In [ ]:
import cv2 as cv 
import numpy as np 
  

# The video feed is read in as a VideoCapture object 
cap = cv.VideoCapture("videoplayback.mp4") 
  
# ret = a boolean return value from getting the frame, 
# first_frame = the first frame in the entire video sequence 
ret, first_frame = cap.read() 
  
# Converts frame to grayscale- less computationally expensive 
prev_gray = cv.cvtColor(first_frame, cv.COLOR_BGR2GRAY) 
  
# Creates an image filled with zero intensities with the same dimensions as the frame 
mask = np.zeros_like(first_frame) 
  
# Sets image saturation to maximum 
mask[..., 1] = 255
  
while(cap.isOpened()): 
      
    # frame = the current frame being projected in the video 
    ret, frame = cap.read() 
      
    # Opens a new window and displays the input frame 
    cv.imshow("input", frame) 
      
    # Converts each frame to grayscale - we previously  
    # only converted the first frame to grayscale 
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY) 
      
    # Calculates dense optical flow by Farneback method 
    flow = cv.calcOpticalFlowFarneback(prev_gray, gray,  None, 0.5,  3,  15,  3,  5,  1.2,  0) 
      
    # Computes the magnitude and angle of the 2D vectors 
    magnitude, angle = cv.cartToPolar(flow[..., 0], flow[..., 1]) 
      
    # Sets image hue according to the optical flow  direction 
    mask[..., 0] = angle * 180 / np.pi / 2
      
    # Sets image value according to the optical flow magnitude (normalized) 
    mask[..., 2] = cv.normalize(magnitude, None, 0, 255, cv.NORM_MINMAX) 
      
    # Converts HSV to RGB (BGR) color representation 
    rgb = cv.cvtColor(mask, cv.COLOR_HSV2BGR) 
      
    # Opens a new window and displays the output frame 
    cv.imshow("dense optical flow", rgb) 
      
    # Updates previous frame 
    prev_gray = gray 
    
    if cv.waitKey(1) & 0xFF == ord('q'): 
        break

# The following frees up resources and closes all windows 
cap.release() 
cv.destroyAllWindows() 